**imports**

In [1]:
import nltk
# nltk.download('stopwords')
import spacy
import PyPDF2
import docx2txt
import pyresparser

**Using Sklearn**

In [2]:
resume = docx2txt.process("D:/fun projects/ATS/Resumes/adithya.docx")

In [3]:
with open("D:/fun projects/ATS/Job_desc/AI_job_desc.txt") as f:
    job_desc = f.read()

In [4]:
text_list = [resume,job_desc]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(text_list)

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
match_matrix = cosine_similarity(X)
print (match_matrix)

[[1.         0.62401323]
 [0.62401323 1.        ]]


In [7]:
match = match_matrix[0][1]
match = match*100
match = round(match, 2)
print("Match Percentage: ", match, "%")

Match Percentage:  62.4 %


In [8]:
if match > 60 :
    print("You are selected")
else:
    print("You are not selected")

You are selected


**Using SpaCy**

In [9]:
nlp = spacy.load('en_core_web_lg')
resume_doc = nlp(resume)
job_desc_doc = nlp(job_desc)
similarity = resume_doc.similarity(job_desc_doc)
match_percentage = similarity * 100

print("Similarity Percentage:", round(match_percentage, 2), "%")

if match_percentage > 60:
    print("You are selected")
else:
    print("You are not selected")

Similarity Percentage: 77.93 %
You are selected


**Using Python resume parsor and transformers**

In [10]:
from pyresparser import ResumeParser
import os

In [11]:
resume = docx2txt.process("D:/fun projects/ATS/Resumes/adithya.docx")

**We will be using NER, in SPACY**

**The Data annotation Tools which we use are :**

     * doccano
     * Labelstudio
     * Inception

In [46]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [47]:
spacy.__version__

'3.6.1'

200 CV used for Training phase

In [48]:
cv_data = json.load(open('D:/fun projects/ATS/ATS/Train/train_data.json', 'r'))
print(len(cv_data))

200


In [74]:
!python -m spacy init fill-config Train\base_config.cfg Train\config.cfg

✔ Auto-filled config with all values
✔ Saved config
Train\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
cv_data

In [71]:
from re import T
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()
    
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
        
        ents=[]
        entity_indices=[]
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                for idx in entity_indices:
                    skip_entity = True
                    break
                if skip_entity==True:
                    continue
                entity_indices = entity_indices + list(range(start, end))
                try:
                    span = doc.char_span(start, end, label=label, alignment_mode='contract')
                except:
                    continue
                
                if span is None:
                    err_data = str([start,end]) + "     " + str(text)+ "\n"
                    file.write(err_data)
                else:
                    ents.append(span)
            try:
                doc.ents = ents
                db.add(doc)
            except:
                pass
    return db
    

In [72]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(cv_data,test_size=0.3)
len(train),len(test)

(140, 60)

In [73]:
file = open('error.txt', 'w')

db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

  6%|▌         | 8/140 [00:00<00:03, 37.38it/s]


UnicodeEncodeError: 'charmap' codec can't encode character '\u27a2' in position 141: character maps to <undefined>